In [ ]:
import topmost
from topmost.data import download_dataset

device = "cuda"  # or "cpu"
dataset_dir = "./datasets/NYT"

In [3]:
download_dataset('NYT', cache_path='./datasets')

https://raw.githubusercontent.com/BobXWu/TopMost/master/data/NYT.zip
Using downloaded and verified file: ./datasets/NYT.zip


In [4]:
# load a dataset
dataset = topmost.data.DynamicDataset(dataset_dir, batch_size=200, read_labels=True, device=device)

all train size:  8254
all test size:  918
all vocab size:  10000
average length: 175.429
num of each time slice:  11 [ 194  265  431  554  744  837  802  884 1283 1400  860]


In [5]:
# create a model
# model = topmost.models.DETM(
#     vocab_size=dataset.vocab_size,
#     num_times=dataset.num_times,
#     train_size=dataset.train_size,
#     train_time_wordfreq=dataset.train_time_wordfreq,
#     device=device,
# )
# model = model.to(device)
# trainer = topmost.trainers.DynamicTrainer(model, epochs=400)

model = topmost.models.CFDTM(
    vocab_size=dataset.vocab_size,
    num_times=dataset.num_times,
    pretrained_WE=dataset.pretrained_WE,
    train_time_wordfreq=dataset.train_time_wordfreq,
)
model = model.to(device)
trainer = topmost.trainers.DynamicTrainer(model, dataset, epochs=800)

top_words, train_theta = trainer.train()

  0%|          | 0/800 [00:00<?, ?it/s]

100%|██████████| 800/800 [27:57<00:00,  2.10s/it]


Notice: Fix the bug of invalid values of coherence models in gensim following [RaRe-Technologies/gensim#3040](https://github.com/RaRe-Technologies/gensim/issues/3040#issuecomment-812913521).

In [13]:
########################### Evaluate ####################################
from topmost import evaluations

# get theta (doc-topic distributions)
train_theta, test_theta = trainer.export_theta()

train_times = dataset.train_times.cpu().numpy()
# compute topic coherence
dynamic_TC = evaluations.dynamic_TC(dataset.train_texts, train_times, dataset.vocab, top_words)
print("dynamic_TC: ", dynamic_TC)

# compute topic diversity
dynamic_TD = evaluations.dynamic_TD(top_words, dataset.train_bow.cpu().numpy(), train_times, dataset.vocab)
print("dynamic_TD: ", dynamic_TD)

# evaluate clustering
results = evaluations.evaluate_clustering(test_theta, dataset.test_labels)
print(results)

# evaluate classification
results = evaluations.evaluate_classification(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
print(results)

======= Time: 0 =======
Topic 0: officer officers police bull man arrested driver grandson shot driving street fatally boy heir bystanders
Topic 1: news horses hbo affidavit scandal british murdoch book sexual abuse sex charges women authorities activities
Topic 2: art neighborhood artist space museum building center roosevelt million queens place apartment century brooklyn sale
Topic 3: slide subscriber planet happens gift messenger ice page frozen reveal islands articles fortunes cold degrees
Topic 4: school schools students education student parents teachers children chicago programs strike teacher district safe lin
Topic 5: doctors home getting hospital wave patients got keep town morning started hour closed medical someone
Topic 6: like want something know care love just people applause man clothing right going police back
Topic 7: museums agency library emirates station plan dhabi climate land authority bridge air water abu project
Topic 8: republican election race elections pres

11it [03:56, 21.49s/it]


dynamic_TC:  0.5394247966925868


100%|██████████| 11/11 [00:00<00:00, 25.93it/s]


dynamic_TD:  0.7042424242424242
{'Purity': 0.6339869281045751, 'NMI': 0.34175043973103136}
{'acc': 0.6023965141612201, 'macro-F1': 0.3720168939430668}
